In [15]:
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split
import matplotlib
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge as Ridge_Reg
from sklearn.linear_model import Lasso as Lasso_Reg
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import r2_score
import xgboost as xgb

### 1. Load the results from the previous models

In [3]:
# load the results from all the models
train_result = pd.read_csv('results/train_results.csv')
test_result = pd.read_csv('results/test_results.csv')
train_xgb = pd.read_csv('results/xgb_train.csv', header = None)
test_xgb = pd.read_csv('results/xgb.csv')

In [6]:
# combine the results for training and testing the ensemble models
x_train_df = pd.concat([train_result, train_xgb], axis = 1)
x_test_df = pd.concat([test_result, test_xgb[[1]]], axis = 1)
# extract the values
x_train = x_train_df.values
x_test = x_test_df.values
# the results from XGboost are converted by log-transformation
x_train[:, 3] = np.log(x_train[:, 3])
x_test[:, 3] = np.log(x_test[:, 3])

In [8]:
# get the y value of the training set
y_train_log = np.loadtxt('data/y_train_log.txt', delimiter=',')
n = len(y_train_log)

### 2. Ensemble models

#### 0. Baseline: average

In [9]:
y_pred_average = np.mean(x_train, axis = 1)

rmse = np.sqrt(np.sum((y_pred_average - y_train_log)**2)/n)
print "The baseline RMSLE is ", rmse

The baseline RMSLE is  0.12173111405


#### 1. Lasso regression

In [10]:
## Lasso regression
# parameters
param = np.power(10.0, range(-4, 5, 1))
# tune lasso regression
model = Lasso_Reg()
grid_model = GridSearchCV(model, param_grid = {'alpha': param}, cv  = 5)
grid_model.fit(x_train, y_train_log)
# best model
lasso = grid_model.best_estimator_
print "The R-squared of the best lasso model is ", grid_model.best_score_
print "The best alpha is ", lasso.get_params()['alpha']

## Prediction
# training
y_pred_train_lasso = cross_val_predict(lasso, x_train, y_train_log, cv = 5)
rmse = np.sqrt(np.sum((y_pred_train_lasso - y_train_log)**2)/n)
print "The model yields RMSLE of ", rmse

The R-squared of the best lasso model is  0.910767719631
The best alpha is  0.001
The model yields RMSLE of  0.119406478031


#### 2. Ridge regression

In [11]:
## Ridge regression
# tune lasso regression using the same parameter list
model = Ridge_Reg()
grid_model = GridSearchCV(model, param_grid = {'alpha': param}, cv  = 5)
grid_model.fit(x_train, y_train_log)
# best model
ridge = grid_model.best_estimator_
print "The R-squared of the best ridge model is ", grid_model.best_score_
print "The best alpha is ", ridge.get_params()['alpha']

## Prediction
# training
y_pred_train_ridge = cross_val_predict(ridge, x_train, y_train_log, cv = 5)
rmse = np.sqrt(np.sum((y_pred_train_ridge - y_train_log)**2)/n)
print "The model yield of RMSLE of ", rmse
# # testing
# ridge.fit(x_train_std, y_train_log)
# y_pred_test_ridge = ridge.predict(x_test_std)

The R-squared of the best ridge model is  0.910605293573
The best alpha is  0.1
The model yield of RMSLE of  0.1195149942


#### 3. Random Forest Regressor
Tune max_depth and min_samples_leaf

In [12]:
## Random Forest Regressor
# parameters
max_depth = range(1, 8, 1)
min_samples_leaf = range(1, 4, 1)

# tune RF regressor
model = RandomForestRegressor(n_estimators=100)
grid_model = GridSearchCV(model, param_grid = {'max_depth': max_depth, 'min_samples_leaf': min_samples_leaf},
                          cv  = 5)
grid_model.fit(x_train, y_train_log)

## Prediction on the training set
RF = grid_model.best_estimator_
y_pred_train_RF = cross_val_predict(RF, x_train, y_train_log, cv = 5)
rmse = np.sqrt(np.sum((y_pred_train_RF - y_train_log)**2)/n)
print "The model yields RMSLE of ", rmse

The model yields RMSLE of  0.121859332638


#### 4. XGBoost

In [16]:
# formatting for xgb
dtrain = xgb.DMatrix(x_train, label=y_train_log)
dtest = xgb.DMatrix(x_test)

# set initial parameters
xgb1 = xgb.XGBRegressor(
 learning_rate =0.1,
 n_estimators=1000,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'reg:linear',
 nthread = 4,
 seed=0)

xgb_param = xgb1.get_xgb_params()

Using the default learning rate of 0.2 to tune the other parameters

In [17]:
# use 5-fold CV
cv_folds = 5
# stop when perfomance does not improve for 50 rounds
early_stopping_rounds = 50

# tune number of trees
cvresult = xgb.cv(xgb_param, dtrain, num_boost_round=xgb_param['n_estimators'], nfold=cv_folds,
    metrics='rmse', early_stopping_rounds=early_stopping_rounds)

print "The model requires {} estimators.". format(cvresult.shape[0])
# update the parameter
n_estimators = cvresult.shape[0]
xgb_param['n_estimators'] = n_estimators
# performance
print "The rmse is ", cvresult['test-rmse-mean'][n_estimators-1]

The model requires 77 estimators.
The rmse is  0.1225666


In [18]:
### Function to tune two parameters of XGBoost
## Input: name and choices of the two parameters, the model and training data
## Output: best score and the two parameters
def two_param_tuning (param_name_1, list_1, param_name_2, list_2, xgb_param, dtrain):
    best_score = 10e4
    best_param_1 = -1
    best_param_2 = -1

    for param_1 in list_1:
        xgb_param[param_name_1] = param_1
        
        for param_2 in list_2:
            xgb_param[param_name_2] = param_2

            cvresult = xgb.cv(xgb_param, dtrain, num_boost_round=xgb_param['n_estimators'], nfold=cv_folds,
                              metrics='rmse')
            score = cvresult['test-rmse-mean'][xgb_param['n_estimators']-1]

            if score < best_score:
                best_score = score
                best_param_1 = param_1
                best_param_2 = param_2
    
    return best_score, best_param_1, best_param_2


### Function to tune one parameter of XGBoost
## Input: name and choices of the parameter, the model and training data
## Output: best score and the parameter
def one_param_tuning (param_name_1, list_1, xgb_param, dtrain):
    best_score = 10e4
    best_param_1 = -1

    for param_1 in list_1:
        xgb_param[param_name_1] = param_1

        cvresult = xgb.cv(xgb_param, dtrain, num_boost_round=xgb_param['n_estimators'], nfold=cv_folds,
                          metrics='rmse')
        score = cvresult['test-rmse-mean'][xgb_param['n_estimators']-1]

        if score < best_score:
            best_score = score
            best_param_1 = param_1
    
    return best_score, best_param_1

**Parameters to tune:**
- max_depth_list and min_child_weight_list
- gamma
- subsample and colsample_bytree

In [36]:
# options
max_depth_list = range(1, 6, 2)
min_child_weight_list = range(1, 6, 2)

best_score, best_max_depth, best_min_child_weight = two_param_tuning('max_depth', max_depth_list, 
                                                                     'min_child_weight', min_child_weight_list,
                                                                     xgb_param, dtrain)

print "The best max_depth is ", best_max_depth
print "The best min_child_weight is ", best_min_child_weight

# update the parameters
xgb_param['max_depth'] = best_max_depth
xgb_param['min_child_weight'] = best_min_child_weight

The best max_depth is  3
The best min_child_weight is  3


In [37]:
gamma_list = [i/100.0 for i in range(0,5)]

best_score, best_gamma = one_param_tuning('gamma', gamma_list, xgb_param, dtrain)

print "The best gamma is ", best_gamma

# update the parameter
xgb_param['gamma'] = best_gamma

The best gamma is  0.0


In [28]:
subsample_list = [i/10.0 for i in range(6,10)]
colsample_bytree_list = [i/10.0 for i in range(4,8)]

best_score, best_subsample, best_colsample_bytree = two_param_tuning('subsample', subsample_list, 'colsample_bytree', colsample_bytree_list,
                                              xgb_param, dtrain)

print "The best subsample is ", best_subsample
print "The best colsample_bytree is ", best_colsample_bytree

# update the parameters
xgb_param['subsample'] = best_subsample
xgb_param['colsample_bytree'] = best_colsample_bytree

The best subsample is  0.7
The best colsample_bytree is  0.5


Fine tune the model

In [29]:
xgb_param['learning_rate'] = 0.01
xgb_param['n_estimators'] = 5000

early_stopping_rounds = 50

cvresult = xgb.cv(xgb_param, dtrain, num_boost_round=xgb_param['n_estimators'], nfold=cv_folds,
    metrics='rmse', early_stopping_rounds=early_stopping_rounds)

print "The model requires {} estimators.". format(cvresult.shape[0])
# update the parameter
n_estimators = cvresult.shape[0]
xgb_param['n_estimators'] = n_estimators
# performance
print "The rmse is ", cvresult['test-rmse-mean'][n_estimators-1]

The model requires 739 estimators.
The rmse is  0.1180122


### 3. Predict on the test set using the best model
The models with the lowest RMSLE are lasso and XGBoost. 

In [13]:
## lasso
lasso.fit(x_train, y_train_log)
y_pred = lasso.predict(x_test)
# clean some outliers
# if the predicted value is beyond the previous results +-0.5, 
# replace by the averaged result
y_test_average = np.mean(x_test, axis = 1)
outliers = (y_pred < np.min(x_test)-0.5) | (y_pred > np.max(x_test) + 0.5)
y_pred[outliers] = y_test_average[outliers] 

# convert to original scale and save
submission = pd.read_csv('data/sample_submission.csv')
submission['SalePrice'] = np.exp(y_pred)
submission.to_csv('results/ensemble.csv', sep = ',', index=False)

In [38]:
## XGboost
xgb1 = xgb.XGBRegressor(
 learning_rate = 0.01,
 n_estimators = 739,
 subsample = 0.7,
 colsample_bytree = 0.5,
 objective= 'reg:linear',
 max_depth = 3,
 min_child_weight = 3,
 nthread = 4,
 seed=0)

# fit the model
xgb1.fit(x_train, y_train_log)
# prediction
y_pred = xgb1.predict(x_test)

# convert to original scale and save
submission['SalePrice'] = np.exp(y_pred)
submission.to_csv('results/ensemble_2.csv', sep = ',', index=False)